# Hoja de trabajo 8 Máquinas Vectoriales de Soporte

repo: https://github.com/Diegoval-Dev/DM-HDT8
- Gerson Ramirez - 22281
- Diego Valenzuela - 22309

In [1]:

import pandas as pd
import numpy as np

df = pd.read_csv("processed_data.csv")

print(f"Dimensiones del dataset: {df.shape}")
df.head()


Dimensiones del dataset: (1460, 77)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,PriceCategory
0,1,60,3,65.0,8450,1,3,3,0,4,...,0,0,0,0,2,2008,8,4,208500,Cara
1,2,20,3,80.0,9600,1,3,3,0,2,...,0,0,0,0,5,2007,8,4,181500,Intermedia
2,3,60,3,68.0,11250,1,0,3,0,4,...,0,0,0,0,9,2008,8,4,223500,Cara
3,4,70,3,60.0,9550,1,0,3,0,0,...,0,0,0,0,2,2006,8,0,140000,Intermedia
4,5,60,3,84.0,14260,1,0,3,0,2,...,0,0,0,0,12,2008,8,4,250000,Cara


## Separación de conjuntos de entrenamiento y prueba

In [4]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["SalePrice", "PriceCategory"])
y = df["PriceCategory"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape, y_train.value_counts(), y_test.value_counts()


((1168, 75),
 (292, 75),
 Intermedia    392
 Economica     390
 Cara          386
 Name: PriceCategory, dtype: int64,
 Intermedia    98
 Cara          97
 Economica     97
 Name: PriceCategory, dtype: int64)

## Exploración de variables y transformaciones para SVM

Las máquinas vectoriales de soporte (SVM) son algoritmos sensibles a la escala de las variables, por lo que es necesario estandarizar los datos antes de entrenar los modelos. Además, las variables categóricas deben ser transformadas a formato numérico mediante codificación one-hot.

En este paso, se inspeccionarán las características de `X_train` para determinar:
- Qué variables son numéricas y cuáles categóricas.
- Si existen valores nulos que deben ser imputados.
- Las transformaciones necesarias para asegurar compatibilidad con los modelos de SVM.

Posteriormente, se construirá un pipeline de preprocesamiento que integre imputación, codificación y escalamiento, utilizando `ColumnTransformer` y `Pipeline` de `sklearn`.

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

numeric_cols = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = X_train.select_dtypes(include=["object", "category"]).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numeric_cols),
    ("cat", categorical_transformer, categorical_cols)
])

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

X_train_processed.shape, X_test_processed.shape


((1168, 75), (292, 75))